In [80]:
#in: picke files with X_train, y_train, X_test, y_test 
#out: picke files with features  for test and train (count_neighbors_train, page_rank_test etc) 
import numpy
import pandas as pd
import networkx as nx
import pickle
import hashlib
from itertools import combinations

from google.colab import drive
drive.mount('/content/gdrive')
path_to_objects = '/content/gdrive/My Drive/UCU-2019-final-project-storage/serialization_objects/'


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
X_train = pickle.load(open(path_to_objects+'X_train.p', 'rb'))
y_train = pickle.load(open(path_to_objects+'y_train.p', 'rb'))
X_test = pickle.load(open(path_to_objects+'X_train.p', 'rb'))
y_test = pickle.load(open(path_to_objects+'y_test.p', 'rb'))

In [0]:
def add_clique_size(X, y):
  G = nx.Graph()
  edges = [tuple(x) for x in X[['question1', 'question2']].values]
  G.add_edges_from(edges)
  with_y = pd.concat([X, y], axis=1)
  map_label = dict(((x[0], x[1]), x[2]) for x in with_y[['question1', 'question2', 'is_duplicate']].values)
  map_clique_size = {}
  cliques = sorted(list(nx.find_cliques(G)), key=lambda x: len(x))
  for cli in cliques:
      for q1, q2 in combinations(cli, 2):
          if (q1, q2) in map_label:
              map_clique_size[q1, q2] = len(cli)
          elif (q2, q1) in map_label:
              map_clique_size[q2, q1] = len(cli)
  X['clique_size'] = X.apply(lambda row: map_clique_size.get((row['question1'], row['question2']), -1), axis=1)



In [0]:
add_clique_size(X_train, y_train)

In [0]:
add_clique_size(X_test, y_test)

In [83]:
X_train.head(5)

,qid1,qid2,question1,question2,clique_size
id,,,,,
243973,356509,356510,How does airbnb screen its host?,When as an Airbnb host can I review my guests?,2
82523,123111,45893,How can I learn hacking for free?,How can I learn hacking for security purposes?,5
373083,41716,2986,How can I speak fluent English with accuracy?,How do I speak English like celebrities?,7
145241,86221,51226,What are the best books for UPSC?,Which are the best books to prepare for IAS exam?,2
227393,336229,302258,Why do smart people have to ask questions on Q...,Why do people ask questions on Quora?,2


In [0]:
pickle.dump(X_train, open(path_to_objects+"X_train_graph.p", 'wb'))
pickle.dump(X_test, open(path_to_objects+"X_test_graph.p", 'wb'))